## Hollywood Case Study

Import libraries

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import sklearn as skl
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

This is the final model and prior to this model, I've build the following four models choosing between Upsamling,(Stratified)K- Fold Cross Validation and  Test- Train Split.
- K- Fold Cross Validation WITH upsampling the class labels
- Test- Train split WITH upsampling the class labels
- K- Fold Cross Validation WITHOUT upsampling the class labels-
- Test- Train split WITHOUT upsampling the class labels.

###### Final Model- Stratified K fold Cross Validation without upsampling, tested on a new set of data extracted from original training sheet.

#### Upsampling-
- The main idea of upsampling is to prevent the model to be more baised towards the majority class labels.
- The 9 class labels present in the training sheet are highly imblanced. So, initially I have built a model by upsampling the minority class labels and balanced them by bringing each class label count to ~250. But, by doing so, the model was overfitting the data.
- For example, the count of class label 9 is just 40 and by upsampling, the 40 rows get duplicated by 6 times to make them 240 and as there is a high chance of duplicated rows are to be present in both train and test data. This leads to over fitting and is not good at all compared to the model being more baised towards majority classes.
- The only solution to this would be gathering more amount of data.

#### Stratified K Fold Cross Validation:
- The data set is too small with only 1200 rows with multiple class. Therefore, Stratified K Fold is chosen as it helps to defend the problem the small sized data set and and deals well with the multiple class labels.

#### Train Test Split
- To validate the results produced by KFold CV, I've used test train split. The accuracy values are almost same in both the ways.

### Final Model

Before building the model, I have split the training sheet into two parts:
- Train(999 rows)
- Test(197 rows)

In [2]:
# Loading the 'Train' file
data= pd.read_excel("Train.xlsx")
data.head()

,id,name,display_name,production_year,movie_sequel,creative_type,source,production_method,genre,language,board_rating_reason,movie_board_rating_display_name,movie_release_pattern_display_name,total,Category
0,70115,"10,000 B.C.","10,000 B.C.",2008,0,Historical Fiction,Original Screenplay,Live Action,Adventure,English,for sequences of intense action and violence,PG-13,Wide,269,6
1,180115,12 Rounds,12 Rounds,2009,0,Contemporary Fiction,Original Screenplay,Live Action,Action,English,for sequences of violence and action,PG-13,Wide,18,2
2,270115,1408,1408,2007,0,Contemporary Fiction,Based on Fiction Book/Short Story,Live Action,Horror,English,for thematic material including disturbing seq...,PG-13,Wide,131,5
3,630115,2010 Oscar Shorts,2010 Oscar Shorts,2009,1,Multiple Creative Types,Compilation,Multiple Production Methods,Multiple Genres,English,International - to be excluded,Not Rated,Limited,1,1
4,640115,2012,2012,2009,0,Science Fiction,Original Screenplay,Live Action,Action,English,for intense disaster sequences and some language,PG-13,Wide,767,9


In [3]:
print("Shape of Train Data:", data.shape)

Shape of Train Data: (999, 15)


In [4]:
#Check for missing values
data.isnull().sum()

id                                    0
name                                  0
display_name                          0
production_year                       0
movie_sequel                          0
creative_type                         0
source                                0
production_method                     0
genre                                 0
language                              0
board_rating_reason                   0
movie_board_rating_display_name       0
movie_release_pattern_display_name    0
total                                 0
Category                              0
dtype: int64

There are no missing values

### Exploratory Data Analysis & Feature selection

In [5]:
data.describe(include="all")

,id,name,display_name,production_year,movie_sequel,creative_type,source,production_method,genre,language,board_rating_reason,movie_board_rating_display_name,movie_release_pattern_display_name,total,Category
count,9.990000e+02,999,999,999.000000,999.000000,999,999,999,999,999,999,999,999,999.000000,999.000000
unique,NaN,999,999,NaN,NaN,9,16,6,13,16,840,6,7,NaN,NaN
top,NaN,No Eres Tu Soy Yo,Love Happens,NaN,NaN,Contemporary Fiction,Original Screenplay,Live Action,Drama,English,International - to be excluded,R,Wide,NaN,NaN
freq,NaN,1,1,NaN,NaN,528,523,911,268,952,69,375,653,NaN,NaN
mean,1.005748e+08,NaN,NaN,2009.098098,0.101101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,108.473473,3.586587
std,4.477719e+07,NaN,NaN,1.410093,0.301614,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,190.364362,1.994547
min,7.011500e+04,NaN,NaN,2007.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,1.000000
25%,6.866012e+07,NaN,NaN,2008.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.000000,2.000000
50%,1.080201e+08,NaN,NaN,2009.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41.000000,3.000000
75%,1.381901e+08,NaN,NaN,2010.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,119.500000,5.000000


- The data contains 15 features out of which 1 is a class label.
- id does not help in building the predictive model.
- Both the features "name" and "display_name" have 999 unique data points which are in the form of string/object data type. Converting all of them into numeric(creating dummy variables) would cost the model a very high dimensionality, which ultimately doesn't help the model and leads to bad one.
- The feature "board_rating_reason" is also irrelevent while we have "movie_board_rating_display_name" derived from the former.
- All the other feature seem to be pretty useful. 

In [5]:
#Dropping the irrelevant/useless features
to_drop= ["id","name","display_name","board_rating_reason"]
data1= data.drop(to_drop, axis=1)

Next step: Investigating the data types and converting them into the appropriate ones.

In [7]:
data1.dtypes

production_year                        int64
movie_sequel                           int64
creative_type                         object
source                                object
production_method                     object
genre                                 object
language                              object
movie_board_rating_display_name       object
movie_release_pattern_display_name    object
total                                  int64
Category                               int64
dtype: object

#### Features that need to be converted into Categorical
- id- int64
- name- object
- display_name- object
- production_year- int64
- movie_sequel- int64- Categorical
- creative_type- object- Categorical
- source- object- Categorical
- production_method- object- Categorical
- genre- object- Categorical
- language- object- Categorical
- board_rating_reason- object
- movie_board_rating_display_name- object- Categorical(Nominal/ Ordinal)
- movie_release_pattern_display_name- object- Categorical
- total- int64
- Category- int64- Categorical

In [6]:
# col=["movie_sequel","creative_type","source","production_method","genre",
#                  "language","movie_board_rating_display_name","movie_release_pattern_display_name"]
# for i in col:
#     data1[i]= data1[i].astype("category", categories = data1[column].unique())

for column in data1.select_dtypes(include=[np.object]).columns:
    data1[column] = data1[column].astype('category', categories = data1[column].unique())

In [7]:
data1.dtypes

production_year                          int64
movie_sequel                             int64
creative_type                         category
source                                category
production_method                     category
genre                                 category
language                              category
movie_board_rating_display_name       category
movie_release_pattern_display_name    category
total                                    int64
Category                                 int64
dtype: object

### One hot encoding: Converting Categorical variables to Numeric

In [8]:
nominal_columns=["movie_sequel","creative_type","source","production_method","genre",
                 "language","movie_board_rating_display_name","movie_release_pattern_display_name"]
df= pd.get_dummies(data=data1, columns=nominal_columns)
df.head() #data frame containing dummy variables for nominal categorical features

,production_year,total,Category,movie_sequel_0,movie_sequel_1,creative_type_Historical Fiction,creative_type_Contemporary Fiction,creative_type_Multiple Creative Types,creative_type_Science Fiction,creative_type_Fantasy,...,movie_board_rating_display_name_PG,movie_board_rating_display_name_G,movie_board_rating_display_name_NC-17,movie_release_pattern_display_name_Wide,movie_release_pattern_display_name_Limited,movie_release_pattern_display_name_Expands Wide,movie_release_pattern_display_name_Oscar Qualifying Run,movie_release_pattern_display_name_Exclusive,movie_release_pattern_display_name_IMAX,movie_release_pattern_display_name_Special Engagement
0,2008,269,6,1,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,2009,18,2,1,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,2007,131,5,1,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,2009,1,1,0,1,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
4,2009,767,9,1,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0


#### Analysis of target column "Category"

Let's count how many number of data points are present in each type of class. So that any imbalance in the target column can be found.

In [10]:
df["Category"].value_counts(dropna= False )

3    207
2    196
4    171
1    146
5    107
6     69
7     51
8     34
9     18
Name: Category, dtype: int64

The class labels are highly imbalanced but as mentioned earlier, upsampling is leading to overfitting. Hence, i've decided to keep the class labels as they are.

### Separating Features and Target in Training Data

In [9]:
features= df.drop("Category", axis=1)
target=df["Category"]

In [10]:
X=features
y=target

In [13]:
X.head()

,production_year,total,movie_sequel_0,movie_sequel_1,creative_type_Historical Fiction,creative_type_Contemporary Fiction,creative_type_Multiple Creative Types,creative_type_Science Fiction,creative_type_Fantasy,creative_type_Kids Fiction,...,movie_board_rating_display_name_PG,movie_board_rating_display_name_G,movie_board_rating_display_name_NC-17,movie_release_pattern_display_name_Wide,movie_release_pattern_display_name_Limited,movie_release_pattern_display_name_Expands Wide,movie_release_pattern_display_name_Oscar Qualifying Run,movie_release_pattern_display_name_Exclusive,movie_release_pattern_display_name_IMAX,movie_release_pattern_display_name_Special Engagement
0,2008,269,1,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,2009,18,1,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,2007,131,1,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,2009,1,0,1,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,2009,767,1,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0


Preparing X and y to fit the scikit learn

In [14]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X=features.as_matrix().astype(np.float)
X= scaler.fit_transform(X)
y=target.values

In [11]:
print("Shape of features X:",X.shape)
print("Shape of target label y:",y.shape)
print("Unique values present in target label:",np.unique(y))

Shape of features X: (999, 77)
Shape of target label y: (999,)
Unique values present in target label: [1 2 3 4 5 6 7 8 9]


In [ ]:
# c,r= y_train.shape
# y_train= y_train.reshape(c,)

## Training the Model

Importing the Classifiers and sklearn CV metrics

In [12]:
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB as NB
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.ensemble import GradientBoostingClassifier as GB
from sklearn.metrics import average_precision_score

### Cross Validation

In [48]:
from sklearn.model_selection import cross_val_score

In [51]:
model_name=['Linear SVC','naive_bayes','RandomForestClassifier','KNeighborsClassifier','GradientBoostingClassifier']
models_list= [LinearSVC(),NB(),RF(),KNN(),GB()]
for i, j in zip(model_name, models_list):
    scores = cross_val_score(j, X, y, cv=5)
    print(i+"--"+ "Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Linear SVC--Accuracy: 0.35 (+/- 0.20)
naive_bayes--Accuracy: 0.27 (+/- 0.05)
RandomForestClassifier--Accuracy: 0.73 (+/- 0.06)
KNeighborsClassifier--Accuracy: 0.97 (+/- 0.02)
GradientBoostingClassifier--Accuracy: 0.98 (+/- 0.01)


### K Fold Cross Validation

In [52]:
from sklearn.model_selection import KFold

In [53]:
kf=KFold(n_splits=5)
model_name=['Linear SVC','naive_bayes','RandomForestClassifier','KNeighborsClassifier','GradientBoostingClassifier']
models_list= [LinearSVC(),NB(),RF(),KNN(),GB()]
for i, j in zip(model_name, models_list):
    scores = cross_val_score(j, X, y, cv=kf)
    print(i+"--"+ "Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Linear SVC--Accuracy: 0.23 (+/- 0.14)
naive_bayes--Accuracy: 0.25 (+/- 0.10)
RandomForestClassifier--Accuracy: 0.74 (+/- 0.11)
KNeighborsClassifier--Accuracy: 0.97 (+/- 0.02)
GradientBoostingClassifier--Accuracy: 0.98 (+/- 0.01)


### Stratified K Fold Cross Validation

StratifiedKFold is a variation of k-fold which returns stratified folds: each set contains approximately the same percentage of samples of each target class as the complete set.

In [45]:
from sklearn.model_selection import StratifiedKFold

In [57]:
skf = StratifiedKFold(n_splits=5)
model_name=['Linear SVC','naive_bayes','RandomForestClassifier','KNeighborsClassifier','GradientBoostingClassifier']
models_list= [LinearSVC(),NB(),RF(),KNN(),GB()]
for i, j in zip(model_name, models_list):
    scores = cross_val_score(j, X, y, cv=skf)
    print(scores)
    print(i+"--"+ "Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
    print()

[0.32682927 0.26368159 0.33668342 0.58585859 0.21428571]
Linear SVC--Accuracy: 0.35 (+/- 0.26)

[0.25853659 0.26368159 0.22613065 0.3030303  0.28061224]
naive_bayes--Accuracy: 0.27 (+/- 0.05)

[0.71219512 0.70646766 0.71356784 0.81313131 0.80102041]
RandomForestClassifier--Accuracy: 0.75 (+/- 0.09)

[0.98536585 0.97512438 0.97487437 0.97979798 0.95918367]
KNeighborsClassifier--Accuracy: 0.97 (+/- 0.02)

[0.97560976 0.97014925 0.97487437 0.98484848 0.9744898 ]
GradientBoostingClassifier--Accuracy: 0.98 (+/- 0.01)



### Hyperparameter Tuning

### Grid Search CV

In [63]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import make_scorer

In [72]:
def RF_param_selection(X, y, nfolds):
    n_estimators = [120,300]
    max_depth=[8,15,None]
    min_samples_split=[2,5,10]
    min_samples_leaf=[1,2,5]
    max_features=["auto", "sqrt" or "log2"]
    param_grid = {'n_estimators': n_estimators,
                 'max_depth':max_depth,
                 'min_samples_split':min_samples_split,
                 'min_samples_leaf':min_samples_leaf,
                 'max_features':max_features}
    grid_search = GridSearchCV(RF(), param_grid, cv=nfolds)
    grid_search.fit(X,y)
    grid_search.best_params_
    grid_search.best_score_
    return{'Best Parameters': grid_search.best_params_, 'Best Score': grid_search.best_score_}

In [73]:
RF_param_selection(X,y,5)

{'Best Parameters': {'max_depth': None,
  'max_features': 'sqrt',
  'min_samples_leaf': 1,
  'min_samples_split': 5,
  'n_estimators': 300},
 'Best Score': 0.8018018018018018}

### Let's check whether the model performs in the same way on unseen data.
- Unseen data is the Test data that I've extracted from the original training sheet.

### Checking on New Data

Loading Test file

In [36]:
test= pd.read_excel("Test.xlsx")
test.head()

,id,name,display_name,production_year,movie_sequel,creative_type,source,production_method,genre,language,board_rating_reason,movie_board_rating_display_name,movie_release_pattern_display_name,total,Category
0,16050115,Bratz,Bratz,2007,0,Fantasy,Based on Game,Live Action,Comedy,English,for thematic elements,PG,Wide,26,3
1,16130115,"Brave One, The",The Brave One,2007,0,Contemporary Fiction,Original Screenplay,Live Action,Drama,English,"for strong violence, language, and some sexuality",R,Wide,70,4
2,16180115,Breach,Breach,2007,0,Dramatization,Based on Real Life Events,Live Action,Thriller/Suspense,English,"for violence, sexual content and language",PG-13,Wide,34,3
3,16430115,Brick Lane,Brick Lane,2008,0,Historical Fiction,Based on Fiction Book/Short Story,Live Action,Drama,English,for some sexuality and brief strong language.,PG-13,Exclusive,4,1
4,16480115,Bride Wars,Bride Wars,2009,0,Contemporary Fiction,Original Screenplay,Live Action,Comedy,English,"for suggestive content, language and some rude...",PG,Wide,115,5


In [44]:
print("Shape of Test data is:", test.shape)

Shape of Test data is: (197, 15)


Dropping the columns from the Test data that are not present in the Trained data.

In [37]:
to_drop= ["id","name","display_name","board_rating_reason"]
test_1= test.drop(to_drop, axis=1)

- Assiging astype.Category to Categorical features
- Converting the Categorical features to Numeric by creating dummy variables

In [38]:
# col=["movie_sequel","creative_type","source","production_method","genre",
#                  "language","movie_board_rating_display_name","movie_release_pattern_display_name"]
# for i in col:
#     test[i]= test[i].astype("category")

for column in test_1.select_dtypes(include=[np.object]).columns:
    test_1[column] = test_1[column].astype('category', categories = test_1[column].unique())

In [39]:
nominal_columns=["movie_sequel","creative_type","source","production_method","genre",
                 "language","movie_board_rating_display_name","movie_release_pattern_display_name"]
test_2= pd.get_dummies(data=test_1, columns=nominal_columns)
test_2.shape

(197, 54)

#### Splitting the Test data into Features and Target 

In [40]:
features_score= test_2.drop("Category",axis=1)
target_score=test_2["Category"]
X_score=features_score
y_score=target_score

In [41]:
print(X_score.shape)
print(y_score.shape)
print(np.unique(y_score))

(197, 53)
(197,)
[1 2 3 4 5 6 7 8 9]


#### Aligning the shape of the test data to match the training data.

In [42]:
# Get missing columns in the training test
missing_cols = set(X.columns ) - set(X_score.columns )
# Add a missing column in test set with default value equal to 0
for c in missing_cols:
    X_score[c] = 0
# Ensure the order of column in the test set is in the same order than in train set
X_score = X_score[X.columns]

In [43]:
print("Shape of training data X:",X.shape)
print("Shape of Test data X_score:",X_score.shape)

Shape of training data X: (999, 77)
Shape of Test data X_score: (197, 77)


Now, we can see that the total number of columns in both test and training data are equal.

### Fitting the training data from "Train file" (999 rows)  and finding the predictions on Test data(197 rows) from "Test file".

In [44]:
from sklearn.metrics import confusion_matrix
from sklearn import metrics

In [58]:
model_name=['Linear SVC','naive_bayes','RandomForestClassifier','KNeighborsClassifier','GradientBoostingClassifier']
models_list= [LinearSVC(),NB(),RF(),KNN(),GB()]
for i, j in zip(model_name, models_list):
    j.fit(X,y)
    y_pred=j.predict(X_score)
    print('accuracy of model:'+ i +' is:',metrics.accuracy_score(y_score, y_pred))
    print()
print()

accuracy of model:Linear SVC is: 0.28426395939086296

accuracy of model:naive_bayes is: 0.2182741116751269

accuracy of model:RandomForestClassifier is: 0.7868020304568528

accuracy of model:KNeighborsClassifier is: 0.949238578680203

accuracy of model:GradientBoostingClassifier is: 0.9746192893401016




In [47]:
model=RF(max_depth= None, max_features= 'sqrt', min_samples_leaf= 1, min_samples_split= 5, n_estimators= 300)
model.fit(X,y)
y_pred=model.predict(X_score)
metrics.accuracy_score(y_score, y_pred)

0.8527918781725888

- KNN classifier performed very badly during Cross Validation but gave good predictions for the unseen data, hence we cannot be sure on this model predictions. Parameter tuning might give us a better estimat on the arrived results.
- Gradient Boosting Classifier has good accuracy on both training and unseen data.

### Final Predictions on Scoring Sheet Data using Gradient Boosting Classifier

In [58]:
#import Scoring Sheet file
scoring_data=pd.read_excel("Scoring Sheet.xlsx")
scoring_data.head()

,id,name,display_name,production_budget,production_year,movie_sequel,creative_type,source,production_method,genre,language,board_rating_reason,movie_board_rating_display_name,movie_release_pattern_display_name
0,7970115,"Avengers, The (2011)",Marvel's The Avengers,225000000,2012,0,Super Hero,Based on Comic/Graphic Novel,Live Action,Adventure,English,for intense sequences of sci-fi violence and a...,PG-13,Wide
1,174540115,Dark Knight Rises The,The Dark Knight Rises,0,2012,1,Super Hero,Based on Comic/Graphic Novel,Live Action,Action,English,"for intense sequences of violence and action, ...",PG-13,Wide
2,149080115,Bond 23,Skyfall,200000000,2012,1,Contemporary Fiction,Based on Fiction Book/Short Story,Live Action,Action,English,"for intense violent sequences throughout, some...",PG-13,Wide
3,136020115,Twilight Saga Breaking Dawn Part 2 The,"The Twilight Saga: Breaking Dawn, Part 2",136200000,2012,1,Fantasy,Based on Fiction Book/Short Story,Live Action,Drama,English,for sequences of violence including disturbing...,PG-13,Wide
4,53500115,"Hobbit, The",The Hobbit: An Unexpected Journey,270000000,2012,1,Fantasy,Based on Fiction Book/Short Story,Animation/Live Action,Adventure,English,for extended sequences of intense fantasy acti...,PG-13,Wide


In [11]:
print("Shape of scoring data is:",scoring_data.shape)

Shape of scoring data is: (91, 14)


In [59]:
# Drop the columns, which are not present in training data
to_drop= ["id","name","display_name","board_rating_reason"]
scoring_data_1= scoring_data.drop(to_drop, axis=1)

Assigning data type category to categorical features and converting them into numeric(dummy variables)

In [60]:
for column in scoring_data_1.select_dtypes(include=[np.object]).columns:
    scoring_data_1[column] = scoring_data_1[column].astype('category', categories = scoring_data_1[column].unique())

In [61]:
nominal_columns=["movie_sequel","creative_type","source","production_method","genre",
                 "language","movie_board_rating_display_name","movie_release_pattern_display_name"]
scoring_data_2= pd.get_dummies(data=scoring_data_1, columns=nominal_columns)
scoring_data_2.shape

(91, 50)

In [62]:
X_ScoringData=scoring_data_2

Aligning the scoring data to match the shape of training data

In [63]:
# Get missing columns in the training test
missing_cols = set(X.columns ) - set(X_ScoringData.columns )
# Add a missing column in test set with default value equal to 0
for c in missing_cols:
    X_ScoringData[c] = 0
# Ensure the order of column in the test set is in the same order than in train set
X_ScoringData = X_ScoringData[X.columns]

In [54]:
print("Shape of training data X:",X.shape)
print("Shape of Scoring data X_scoringData:",X_ScoringData.shape)

Shape of training data X: (999, 77)
Shape of Scoring data X_scoringData: (91, 77)


#### Predictions from Linear SVC

In [55]:
from sklearn import metrics
clf= LinearSVC()
clf.fit(X,y)
target_predicted_SVC=clf.predict(X_ScoringData)

In [26]:
Predictions= pd.DataFrame(target_predicted_SVC).to_excel('prediction_SVC.xlsx')

#### Predictions from GB Classifier

In [56]:
clf= GB()
clf.fit(X,y)
target_predicted_GB=clf.predict(X_ScoringData)

In [28]:
Predictions= pd.DataFrame(target_predicted_GB).to_excel('prediction_GB.xlsx')

#### Predictions from Naive Bayes

In [29]:
clf= NB()
clf.fit(X,y)
target_predicted_NB=clf.predict(X_ScoringData)

In [30]:
Predictions= pd.DataFrame(target_predicted_NB).to_excel('prediction_NB.xlsx')

#### Predictions from Random Forest Classifier

In [65]:
model=RF(max_depth= None, max_features= 'sqrt', min_samples_leaf= 1, min_samples_split= 5, n_estimators= 300)
model.fit(X,y)
target_predicted_RF=model.predict(X_ScoringData)
Predictions= pd.DataFrame(target_predicted_RF).to_excel('predictions_RF.xlsx')

- Both Linear SVC and Gradient Boosting Classifiers gave same prediction outputs where as the predictions of Naive Bayes are completely different.
- I've used the 2 best models and 1 worst model on predicting final outputs to check the validity of the model i.e., to see whether they predict different outputs or not.